In [1]:
import anndata as ad
import csv
import gpzoo.pri_experiment_utilities as putil
import matplotlib.pyplot as plt
import numpy as np
import random
import scanpy as sc
import scipy.sparse as sparse
import squidpy as sq
import tensorflow as tf
import time
import torch
import torch.nn.utils as nn_utils

from anndata import AnnData
from contextlib import suppress
from copy import deepcopy
from gpzoo.gp import SVGP
from gpzoo.kernels import NSF_RBF
from gpzoo.likelihoods import NSF2
from gpzoo.utilities import (add_jitter, anndata_to_train_val, deviancePoisson, dims_autocorr, regularized_nmf, rescale_spatial_coords, scanpy_sizefactors)
from os import path
from scanpy import read_h5ad
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
from sklearn.model_selection import ParameterGrid, KFold
from squidpy.gr import spatial_autocorr, spatial_neighbors
from tensorflow import constant
from tensorflow.data import Dataset
from torch import distributions, nn, optim
from tqdm import tqdm

/engelhardt/home/pshrestha/vnngp/GPzoo/gpzoo/pri_experiment_utilities.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/engelhardt/home/pshrestha/anaconda3/envs/vnngp2/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 13.0.0. Please consider upgrading.
  warnings.warn(
2024-09-12 10:29:49.056892: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-12 10:29:49.068601: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has alre

In [2]:
print("CUDA available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())
print("Current GPU: ", torch.cuda.current_device())
print("GPU Name: ", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available:  True
Number of GPUs:  2
Current GPU:  0
GPU Name:  NVIDIA A30


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [5]:
X, Y = putil.load_visium()
Y = Y.T

In [4]:
random.seed(256)
root_path = '/engelhardt/home/pshrestha/vnngp/'
save_path = path.join(root_path, 'results/visium/nnnsfh')

In [ ]:
# inducing points always untrained
# don't train V, initialize it as the median/mean of the expression
# filter an define optimizer again after changing gradients
model.sf.prior.Z.requires_grad=False
model.cf.prior.scale_pf = 1e-1
model.V.requires_grad=False

# 0. train both variances (3 hrs) 
model.cf.prior.scale.requires_grad = True
model.sf.prior.Lu.requires_grad=True

model.sf.prior.kernel.lengthscale.requires_grad = False
model.sf.prior.kernel.sigma.requires_grad = False
model.sf.prior.mu.requires_grad=False
model.sf.W.requires_grad=False
model.cf.W.requires_grad=False
model.cf.prior.mean.requires_grad=False



# 1. train mean and variance (spatial) until convergence  (6 hrs)
model.sf.prior.Lu.requires_grad=True
model.sf.prior.mu.requires_grad=True

model.cf.prior.mean.requires_grad=False
model.cf.prior.scale.requires_grad = False
model.sf.prior.kernel.lengthscale.requires_grad = False
model.sf.prior.kernel.sigma.requires_grad = False
model.sf.W.requires_grad=False
model.cf.W.requires_grad=False


# 2. train mean and variance until convergence + non-spatial variance (3 hrs)
model.sf.prior.Lu.requires_grad=True
model.sf.prior.mu.requires_grad=True
model.cf.prior.scale.requires_grad = True

model.cf.prior.mean.requires_grad=False
model.cf.prior.scale.requires_grad = False
model.sf.prior.kernel.lengthscale.requires_grad = False
model.sf.prior.kernel.sigma.requires_grad = False
model.sf.W.requires_grad=False
model.cf.W.requires_grad=False


# 3. train mean, variance and loadings spatial + non-spatial variance (6hrs)
model.sf.prior.Lu.requires_grad=True
model.sf.prior.mu.requires_grad=True
model.cf.prior.scale.requires_grad = True
model.sf.W.requires_grad=True

model.cf.prior.mean.requires_grad=False
model.cf.prior.scale.requires_grad = False
model.sf.prior.kernel.lengthscale.requires_grad = False
model.sf.prior.kernel.sigma.requires_grad = False
model.cf.W.requires_grad=False

# 4. train non-spatial mean, variance and loadings (6hrs)
model.cf.prior.mean.requires_grad=True
model.cf.prior.scale.requires_grad = True
model.cf.prior.scale.requires_grad = True
model.cf.W.requires_grad=True

model.sf.prior.Lu.requires_grad=False
model.sf.prior.mu.requires_grad=False
model.sf.W.requires_grad=False
model.sf.prior.kernel.lengthscale.requires_grad = False
model.sf.prior.kernel.sigma.requires_grad = False

# 5. train everything (6hrs)
model.cf.prior.mean.requires_grad=True
model.cf.prior.scale.requires_grad = True
model.cf.prior.scale.requires_grad = True
model.cf.W.requires_grad=True
model.sf.prior.Lu.requires_grad=True
model.sf.prior.mu.requires_grad=True
model.sf.W.requires_grad=True
model.sf.prior.kernel.lengthscale.requires_grad = False
model.sf.prior.kernel.sigma.requires_grad = False

In [6]:
kwargs = {
                    'L': 15, 
                    'M': len(X), # use total number of inducing points
                    'sigma': 1.0, 
                    'lengthscale': 1.0, 
                    'jtr': 1e-2, # no lower
                    'batch_size': 128,
                    'lr': 0.001,
                    'rs': 256,
                    'lkzz_build': 1,
                    'model': 'VNNGP',
                    'L1_jitter': 1e-1, # no lower
                    'iterations': 5000,
                    'K': 8,
                }

In [7]:
nmf_save_path = path.join(root_path, 'results/visium/nmf/')
factors_path = path.join(nmf_save_path, f"nmf_factors_iter=1000_rs=256_L={kwargs['L']}.npy")
loadings_path = path.join(nmf_save_path, f"nmf_loadings_iter=1000_rs=256_L={kwargs['L']}.npy")
factors = np.load(factors_path)
loadings = np.load(loadings_path)

In [8]:
moran_idx, moranI = putil.dims_autocorr(factors, np.array(X))
factors = factors[:, moran_idx]
loadings = loadings[:, moran_idx]

In [9]:
model = putil.build_model_hybrid(np.array(X), np.array(Y).T, loadings=loadings, factors=factors, kwargs=kwargs)
losses, means, scales, final_time = [], [], [], 0

In [14]:
model.cf.prior.scale

Parameter containing:
tensor([[0.7512, 0.1662, 0.6926,  ..., 0.2859, 0.4108, 0.5374],
        [0.2251, 0.5524, 0.4697,  ..., 0.7516, 0.3228, 0.0940],
        [0.0101, 0.6810, 0.9064,  ..., 0.2430, 0.5382, 0.0654],
        ...,
        [0.7334, 0.5678, 0.2261,  ..., 0.1728, 0.4853, 0.5088],
        [0.7637, 0.7165, 0.6401,  ..., 0.0384, 0.9027, 0.4810],
        [0.0564, 0.6255, 0.1172,  ..., 0.9951, 0.3971, 0.2547]],
       device='cuda:0', requires_grad=True)

In [1]:
model.sf.prior.kernel.lengthscale.requires_grad = False
model.sf.prior.Z.requires_grad=False
model.sf.prior.mu.requires_grad=False
model.sf.prior.Lu.requires_grad=False
model.sf.W.requires_grad=False
model.cf.W.requires_grad=True
model.cf.prior.mean.requires_grad=True
model.cf.prior.scale_pf = 1e-1
model.V.requires_grad=True

NameError: name 'model' is not defined